In [ ]:
import nltk

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd

In [ ]:
fake = pd.read_csv("Fake.csv")
genuine = pd.read_csv("True.csv")

In [ ]:
display(fake.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB


None

In [ ]:
display(genuine.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB


None

In [ ]:
fake['target'] = 1
genuine['target']=0

In [ ]:
display(fake.head())

,title,text,subject,date,target
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [ ]:
display(genuine.head())

,title,text,subject,date,target
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


In [ ]:
data = pd.concat([fake,genuine],axis=0)

In [ ]:
data = data.reset_index(drop=True)

In [ ]:
data = data.drop(['subject','date','title'],axis=1)

In [ ]:
print(data.columns)

Index(['text', 'target'], dtype='object')


**Tokenization**

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
data['text'] = data['text'].apply(word_tokenize)

In [ ]:
print(data.head())

                                                text  target
0  [Donald, Trump, just, couldn, t, wish, all, Am...       1
1  [House, Intelligence, Committee, Chairman, Dev...       1
2  [On, Friday, ,, it, was, revealed, that, forme...       1
3  [On, Christmas, day, ,, Donald, Trump, announc...       1
4  [Pope, Francis, used, his, annual, Christmas, ...       1


**Stemming**

In [ ]:
from nltk.stem.snowball import SnowballStemmer
porter = SnowballStemmer('english')

In [ ]:
def stem_it(text):
  return [porter.stem(word) for word in text]

In [ ]:
data['text'] = data['text'].apply(stem_it)

In [ ]:
print(data.head())

                                                text  target
0  [donald, trump, just, couldn, t, wish, all, am...       1
1  [hous, intellig, committe, chairman, devin, nu...       1
2  [on, friday, ,, it, was, reveal, that, former,...       1
3  [on, christma, day, ,, donald, trump, announc,...       1
4  [pope, franci, use, his, annual, christma, day...       1


**Stop word removal**

In [ ]:
def stop_it(t):
  dt = [word for word in t if len(word)>2]
  return dt

In [ ]:
data['text'] = data['text'].apply(stop_it)

In [ ]:
print(data.head())

                                                text  target
0  [donald, trump, just, couldn, wish, all, ameri...       1
1  [hous, intellig, committe, chairman, devin, nu...       1
2  [friday, was, reveal, that, former, milwauke, ...       1
3  [christma, day, donald, trump, announc, that, ...       1
4  [pope, franci, use, his, annual, christma, day...       1


In [ ]:
data['text'] = data['text'].apply(' '.join)

**Splitting**

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(data['text'],data['target'],test_size=0.25)
display(X_train.head())
print('\n')
display(y_train.head())

8238     onc broken record alway broken record least th...
14099    portland state univers held second student for...
33933    reuter democrat senat missouri were the 20th h...
42007    berlin reuter frauk petri the former co-lead t...
26679    washington reuter the u.s. senat will again co...
Name: text, dtype: object

8238     1
14099    1
33933    0
42007    0
26679    0
Name: target, dtype: int64

**Vectorization**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
my_tfidf = TfidfVectorizer(max_df=0.7)

tfidf_train = my_tfidf.fit_transform(X_train)
tfidf_test = my_tfidf.transform(X_test)

In [ ]:
print(tfidf_train)

  (0, 32368)	0.05969934968547009
  (0, 41050)	0.02785467999550026
  (0, 31433)	0.029302921897109913
  (0, 63264)	0.059933567370148454
  (0, 9533)	0.02521373180458052
  (0, 21416)	0.0472193751692784
  (0, 69125)	0.05424789894422427
  (0, 69467)	0.02450071490243632
  (0, 62628)	0.03671954142171712
  (0, 64709)	0.031531975224129885
  (0, 89132)	0.01895996595261787
  (0, 57735)	0.05507854508980408
  (0, 60812)	0.025091684388686995
  (0, 17900)	0.03264659214516227
  (0, 86853)	0.07269537371388926
  (0, 49628)	0.022301883564012782
  (0, 50346)	0.03004194299599772
  (0, 38525)	0.02884528944832806
  (0, 59357)	0.06167419575160727
  (0, 26690)	0.03267393968614865
  (0, 18986)	0.03943141123123445
  (0, 88830)	0.019678342507576028
  (0, 61440)	0.02037782227371221
  (0, 35336)	0.09023660830430465
  (0, 88954)	0.05000826654281249
  :	:
  (33672, 33632)	0.1681016394823251
  (33672, 22499)	0.22786728445778007
  (33672, 90095)	0.16346829599637283
  (33672, 33133)	0.43392245146881675
  (33672, 91111)	0

**Logistic regression**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
model_1 = LogisticRegression (max_iter=900)
model_1.fit(tfidf_train, y_train)
pred_1 = model_1.predict(tfidf_test)
cr1 = accuracy_score(y_test, pred_1)
print(cr1*100)

98.79732739420936


**Passive aggressive classfifier**

In [ ]:
from sklearn.linear_model import PassiveAggressiveClassifier
model= PassiveAggressiveClassifier (max_iter=50)
model.fit(tfidf_train, y_train)

PassiveAggressiveClassifier(max_iter=50)

In [ ]:
y_pred = model.predict(tfidf_test)
accscore = accuracy_score(y_test, y_pred)
print('The accuracy of prediction is ',accscore*100)

The accuracy of prediction is  99.69710467706014


In [ ]:
def preprocess_text(text):
    tokens = word_tokenize(text)
    stemmed_tokens = [porter.stem(word) for word in tokens]
    processed_tokens = [word for word in stemmed_tokens if len(word) > 2]
    processed_text = ' '.join(processed_tokens)
    return processed_text

input_text = "In a highly anticipated speech, Vice President Kamala Harris addressed the United Nations General Assembly, reaffirming America's commitment to global cooperation and multilateralism. Harris emphasized the importance of addressing pressing global challenges such as climate change, COVID-19, and economic inequality through collective action. The speech marked a significant moment for Harris, who became the first female vice president of the United States and the highest-ranking woman in U.S. history. Her remarks underscored the Biden administration's efforts to reengage with the international community after years of strained relations under the previous administration"
preprocessed_input = preprocess_text(input_text)
tfidf_input = my_tfidf.transform([preprocessed_input])
prediction = model.predict(tfidf_input)
print("Predicted target:", prediction[0])
if(prediction[0] == 0):
  print("The news is most likely real")
else:
  print("The news is most likely fake")


Predicted target: 0
The news is most likely real
